# Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt   

# Load Data

In [ ]:
# Load data from Excel file
data = pd.read_excel('ECG Datasets.xlsx', header=None).values
# data = pd.read_excel('Temperature Dataset.xlsx', header=None).values

# Normalize the input data
for ii in range(4): 
    data[:, ii] = data[:, ii] / np.max(data[:, ii]) 

# Set parameters

In [ ]:
# Set parameters
percent_train = 0.75

num_data = data.shape[0] 
num_train = round(num_data * percent_train)
num_test = num_data - num_train

n1 = 3 #input-size
n2 = 100
n3 = 20
n4 = 1

eta = 0.01

epoch = 50
mse_train = np.zeros(epoch)
mse_test = np.zeros(epoch)
output_data_train = np.zeros(epoch)
output_data_test = np.zeros(epoch)


a = -1
b = 1

# Initialize weights
w1 = np.random.uniform(a, b, size=(n2, n1))
w2 = np.random.uniform(a, b, size=(n3, n2))
w3 = np.random.uniform(a, b, size=(n4, n3))

# Initialize weights
bias1 = np.random.uniform(a, b, size=(n2, 1))
bias2 = np.random.uniform(a, b, size=(n3, 1))
bias3 = np.random.uniform(a, b, size=(n4, 1))

w_par = np.zeros((num_train, n2 * n1 + n2 * 1 + n3 * n2 + n3 * 1 + n4 * n3 + n4 * 1)) # k * p : num-of-inputs * num-of-parameters
I = np.eye(n2 * n1 + n2 * 1 + n3 * n2 + n3 * 1 + n4 * n3 + n4 * 1)

# Train and Test

In [ ]:
for t in range(epoch):
    error = np.zeros(num_train).reshape(-1,1) 
    
    for i in range(num_train):
        input_data = data[i, :3].reshape(-1,1)
        net1 = w1 @ input_data + bias1
        o1 = 1 / (1 + np.exp(-net1)) 
        net2 = w2 @ o1 + bias2 
        o2 = 1 / (1 + np.exp(-net2))
        net3 = w3 @ o2
        o3 = net3 + bias3
        target = data[i, 3]
        error[i] = target - o3

        t1 = (o2 * (1 - o2)).reshape(-1,1)

        A = np.diag(t1.flatten())

        t2 = o1 * (1 - o1).reshape(-1,1)
        B = np.diag(t2.flatten())
        
        pw1 = -1 * (w3 @ A @ w2 @ B).T @ input_data.T
        pw2 = -1 * ((w3 @ A).T @ o1.T)
        pw3 = -1 * 1 * o2.T
        
        pb1 = -1 * (w3 @ A @ w2 @ B).T 
        pb2 = -1 * (w3 @ A).T
        pb3 = -1 * 1 * np.ones(n4)
        
        a = pw1.flatten()
        b = pb1.flatten()
        c = pw2.flatten()
        d = pb2.flatten()
        e = pw3.flatten()
        f = pb3.flatten()
        
        w_par[i, :] = np.concatenate((a, b, c, d, e, f))


    a1 = w1.reshape(-1,1)
    b1 = bias1.reshape(-1,1)
    c1 = w2.reshape(-1,1)
    d1 = bias2.reshape(-1,1)
    e1 = w3.reshape(-1,1)
    f1 = bias3.reshape(-1,1)
    

    w_par1 = np.concatenate((a1, b1, c1, d1, e1, f1)).reshape(-1,1)

    miu = (error.T @ error).flatten()[0]
    
    w_par1 = w_par1 - np.linalg.inv((w_par.T @ w_par) + miu * I) @ w_par.T @ error 
    
    a2 = w_par1[:n2 * n1].reshape((n2, n1))
    b2 = w_par1[n2 * n1: n2 * n1 + n2 * 1].reshape((n2, 1))
    c2 = w_par1[ n2 * n1 + n2 * 1: n2 * n1 + n2 * 1 + n3 * n2].reshape((n3, n2))
    d2 = w_par1[ n2 * n1 + n2 * 1 + n3 * n2: n2 * n1 + n2 * 1 + n3 * n2 + n3 * 1].reshape((n3, 1))
    e2 = w_par1[ n2 * n1 + n2 * 1 + n3 * n2 + n3 * 1: n2 * n1 + n2 * 1 + n3 * n2 + n3 * 1 + n4 * n3].reshape((n4, n3))
    f2 = w_par1[ n2 * n1 + n2 * 1 + n3 * n2 + n3 * 1 + n4 * n3: n2 * n1 + n2 * 1 + n3 * n2 + n3 * 1 + n4 * n3 + n4 * 1].reshape((n4, 1))

    w1 = a2
    bias1 = b2
    w2 = c2
    bias2 = d2
    w3 = e2
    bias3 = f2
    
#     print('w1: ',w1.shape,'bias1: ',bias1.shape,'w2: ',w2.shape,'bias2: ',bias2.shape,'w3: ',w3.shape,'bias3: ',bias3.shape)
    error_data_train = np.zeros(num_train)
    output_data_train = np.zeros(num_train) 
    
    for i in range(num_train):
        input_data = data[i, :3].reshape(-1,1)
#         print('input_data:',input_data.shape)
        net1 = w1 @ input_data + bias1
#         print('net1: ',net1.shape)
        o1 = 1 / (1 + np.exp(-net1))
#         print('o1: ', o1.shape)
        net2 = w2 @ o1 + bias2
#         print('net2: ', net2.shape)
        o2 = 1 / (1 + np.exp(-net2))
#         print('o2: ',o2.shape)
        net3 = w3 @ o2 + bias3
#         print('net3: ',net3.shape)
        o3 = net3
#         print('o3: ',o3.shape)
        target = data[i, 3]
        output_data_train[i] = o3
        error_data_train[i] = target - o3

    mse_train[t] = np.mean(error_data_train ** 2)

    error_data_test = np.zeros(num_test)
    output_data_test = np.zeros(num_test)
    
    for i in range(num_test):
        input_data = data[num_train + i, :3].reshape(-1,1)
        net1 = w1 @ input_data + bias1
        o1 = 1 / (1 + np.exp(-net1))
        net2 = w2 @ o1 + bias2
        o2 = 1 / (1 + np.exp(-net2))
        net3 = w3 @ o2 + bias3
        o3 = net3
        target = data[num_train + i, 3]
        output_data_test[i] = o3
        error_data_test[i] = target - o3

    mse_test[t] = np.mean(error_data_test ** 2)
    
    # Plotting the training data and output
    plt.figure(figsize=(10, 8))

    plt.subplot(2, 2, 1)
    plt.plot(data[:num_train, 3])
    plt.plot(output_data_train, 'r', linewidth=1)
    plt.xlabel('Train Data')
    plt.ylabel('Output')
    plt.legend(['Actual', 'Predicted'])

    # Plotting the training MSE
    plt.subplot(2, 2, 2)
    plt.semilogy(np.arange(1, t + 1), mse_train[:t])
    plt.xlabel('Epoch')
    plt.ylabel('MSE Train')

    # Plotting the test data and output
    plt.subplot(2, 2, 3)
    plt.plot(data[num_train:num_data, 3])
    plt.plot(output_data_test, 'r', linewidth=1)
    plt.xlabel('Test Data')
    plt.ylabel('Output')
    plt.legend(['Actual', 'Predicted'])

    # Plotting the test MSE
    plt.subplot(2, 2, 4)
    plt.semilogy(np.arange(1, t + 1), mse_test[:t])
    plt.xlabel('Epoch')
    plt.ylabel('MSE Test')
    
    print('Epoch: {} \t'.format(t+1))
    print('MSE_train: {:.4f} \t, MSE_Test: {:.4f}'.format(mse_train[t], mse_test[t]))
    
    plt.tight_layout()
    plt.show()
    
    print("\n\033[1;m" + "*" * 125)


In [ ]:
plt.figure(2)
m_train , b_train = np.polyfit(data[:num_train, 3], output_data_train, 1)    
plt.scatter(data[:num_train, 3], output_data_train,facecolors='none',edgecolors='#104E8B')
plt.plot(data[:num_train, 3], m_train*data[:num_train, 3]+b_train,'r')
# plt.plot(data[:num_train, 3], output_data_train, label='Regression Line', color='red')
plt.title('Regression Train') 

plt.figure(3)
m_test , b_test = np.polyfit(data[num_train:num_data, 3], output_data_test, 1)  
plt.scatter(data[num_train:num_data, 3], output_data_test,facecolors='none',edgecolors='#104E8B')
plt.plot(data[num_train:num_data, 3], m_test*data[num_train:num_data, 3]+b_test,'r')
plt.title('Regression Test')
 

mse_train_result = mse_train[-1]
mse_test_result = mse_test[-1]

print("Final MSE on Train Data:", mse_train_result)
print("Final MSE on Test Data:", mse_test_result)